# Librerías

In [33]:
import h5py
import numpy as np
import pandas as pd

In [9]:
from pydeseq2.dds import DeseqDataSet
from pydeseq2.ds import DeseqStats
from pydeseq2.utils import load_example_data

# Lectura de datos
La prueba se realizará únicamente con el dataset de

In [15]:
dataLiver = h5py.File('../Tests-GMM/data/Small_Datasets/' + 'HumanLiver_counts_top5000.h5')
raw_data = {'X': np.array(dataLiver['X']), 'y': np.array(dataLiver['Y'])}

In [21]:
unique, counts = np.unique(raw_data['y'], return_counts=True)
cluster_max = unique[np.argmax(counts)]

y = np.where(raw_data['y'] == cluster_max, 1, 0)
y

array([0, 0, 0, ..., 0, 0, 0])

In [37]:
dds = DeseqDataSet(
    counts = raw_data['X'],
    metadata = pd.DataFrame(y, columns = ['condition']),
    design_factors = "condition"
)

c:\Users\melir\anaconda3\envs\env_scRNA\lib\site-packages\anndata\_core\anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


In [39]:
# Size factors
dds.fit_size_factors()

dds.obsm["size_factors"]

Fitting size factors...
C:\Users\melir\AppData\Local\Temp\ipykernel_10048\977234819.py:2: RuntimeWarning: Every gene contains at least one zero, cannot compute log geometric means. Switching to iterative mode.
  dds.fit_size_factors()
Fitting dispersions...
... done in 324.98 seconds.

Fitting MAP dispersions...
... done in 247.51 seconds.



Retrieving notices: ...working... done
Solving environment: ...working... failed with initial frozen solve. Retrying with flexible solve.
Solving environment: ...working... failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: ...working... failed with initial frozen solve. Retrying with flexible solve.
Solving environment: ...working... 
Found conflicts! Looking for incompatible packages.
This can take several minutes.  Press CTRL-C to abort.
failed



The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - pytorch/win-64::torchaudio==0.13.1=py310_cpu
  - pytorch/win-64::torchvision==0.14.1=py310_cpu

The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - pytorch/win-64::torchaudio==0.13.1=py310_cpu
  - pytorch/win-64::torchvision==0.14.1=py310_cpu

Examining zipp: 100%|█████████▉| 218/219 [12:06<00:12, 12.58s/it]  
                                                                 

Determining conflicts:   0%|          | 0/219 [00:00<?, ?it/s]
Examining conflict for numba umap-learn scanpy pynndescent:   0%|          | 0/219 [00:00<?, ?it/s]
Examining conflict for numba contourpy numpy scanpy networkx scikit-learn umap-learn patsy pynndescent scipy seaborn torchvision mkl_random statsmodels torchaudio numpy-base anndata mkl_fft mkl-service pandas matplotlib-base h5py:   0%|         

KeyboardInterrupt: 

In [ ]:
# Genewise dispersion
dds.fit_genewise_dispersions()

dds.varm["genewise_dispersions"]

In [ ]:
dds.fit_dispersion_trend()
dds.uns["trend_coeffs"]
dds.varm["fitted_dispersions"]

In [ ]:
dds.fit_dispersion_prior()
print(
    f"logres_prior={dds.uns['_squared_logres']}, sigma_prior={dds.uns['prior_disp_var']}"
)

In [ ]:
dds.fit_MAP_dispersions()
dds.varm["MAP_dispersions"]
dds.varm["dispersions"]

In [ ]:
dds.fit_LFC()
dds.varm["LFC"]

In [ ]:
dds.calculate_cooks()
if dds.refit_cooks:
    # Replace outlier counts
    dds.refit()

In [ ]:
with open("dds_detailed_pipe.pkl", "wb") as f:
        pkl.dump(dds, f)

In [ ]:
stat_res = DeseqStats(dds, alpha=0.05, cooks_filter=True, independent_filter=True)


In [ ]:
stat_res.run_wald_test()
stat_res.p_values

In [ ]:
if stat_res.cooks_filter:
    stat_res._cooks_filtering()
stat_res.p_values

In [ ]:
if stat_res.independent_filter:
    stat_res._independent_filtering()
else:
    stat_res._p_value_adjustment()

stat_res.padj

In [ ]:
stat_res.summary()


In [ ]:
with open("stat_results_detailed_pipe.pkl", "wb") as f:
        pkl.dump(stat_res, f)

In [ ]:
stat_res.lfc_shrink(coeff="condition_B_vs_A")
